In [3]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine,text
import uuid
from datetime import datetime

In [ ]:
#CREATING CONNECTION TO SQL SERVER


conn = pyodbc.connect(
    "DRIVER=OBDC Driver 17 for SQL Server";
    "SERVER=DESKTOP-86E7MBU\\SQLEXPRESS";
    "DATABASE=master";
    "Trusted_Connection=yes"
    )

conn.autocommit=True
cursor = conn.cursor()

#CREATING DATABASE IN SERVER

cursor.execute("""
    IF EXISTS (SELECT 1 FROM sys.databases WHERE name='NWindDatawareHouse')
        BEGIN
            ALTER DATABASE NWindDatawareHouse SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
            DROP DATABASE NWindDatawareHouse
        END;
    CREATE DATABASE NWindDatawareHouse;
""")

cursor.close()
conn.close()
print("DataBase Created Successfully!")



In [ ]:
#CREATE SCHEMAS IN DATABASE

engine = create_engine(
    "mssql+pyodbc://DESKTOP-86E7MBU\\SQLEXPRESS/NWindDatawareHouse?driver=ODBC+Driver+17+for+SQL+Server",
    isolation_level="AUTOCOMMIT"
    )

schemas = ['gold','silver','bronze']

with engine.connect() as conn:
    for schema in schemas:
        conn.execute(text(f"""
            IF NOT EXISTS (SELECT 1 FROM sys.schemas WHERE name='{schema}')
            BEGIN
                EXEC('CREATE SCHEMA {schema}')
            END;
            """))
            
print("Schemas Created Successfully!")





In [18]:
#EXECUTION OF STORED PROCEDURES

engine = create_engine(
    "mssql+pyodbc://DESKTOP-86E7MBU\\SQLEXPRESS/NWindDatawareHouse?driver=ODBC+Driver+17+for+SQL+Server",
    isolation_level="AUTOCOMMIT"
    )

#ADD METADATA
load_time = datetime.now()

steps = [
    "bronze.load_bronze",
    "silver.load_silver",
    "gold.load_gold"
]

with engine.connect() as conn:
    for step in steps:
        try:
            print(f"▶ Running {step}")
            conn.execute(text(f"EXEC {step}"))


        except Exception as e:
            print("Loading Tables Failed!")
            print(f"error : {e}")
            raise



▶ Running bronze.load_bronze
▶ Running silver.load_silver
▶ Running gold.load_gold
